# Explainer Notebook for Harry Potter Network Analysis

This explainer notebook contains the code and detailed analysis about our work on Harry Potter network.   
The webside for presenting our work is here(<font color=red>the link</font>).

<font color=red>Maybe some more general information here</font>
    

### 1. Motivation 
<!--
* What is your dataset?
* Why did you choose this/these particular dataset(s)?
* What was your goal for the end user's experience?
-->

#### Harry Potter Network Dataset

Our dataset is the characters from Harry Potter Novel, including character relationships, character attributes, characer stories.
We got the dataset using API from Fandom pages. Also, J.K rolling’s books will be used as our data source as well.    
<font color=orange>(not enough, we build this dataset by blabla, what source we use, how we get things done. 
simple discrption of the dataset , includeing: amount of nodes, edges, size of files.
the strength(big, contains lots of characters) ;  node attributes
some limitations: the potential problems of this dataset)</font>

#### Why we choose this dataset?

**First of all**, this dataset has a very complete structure of character relations, which is a perfct material for social network analysis.
**Moreover**, there are some particularly interesting studies of the relationship between universes, such as those between wizards and Muggles (non-wizards), from which we hope to derive the new X-degree of separation for whole universe.
**Also**, the division of the four major colleges systems within the Hogwarts also caught our attention, and we wanted to find the attributes and patterns so that everyone could know which house they were assigned to.
**Finally**, the Harry Potter novel accompanies almost everyone's childhood, most of people are very familiar with the characters and stories, hence, it will be easier to understand the outcomes of our exploration. And it would be exciting to explore new discoveries from familiar stories and relationships.

<font color=orange>需要更有逻辑，现在这个非常散，这个可以做完后面的再来写</font>


#### Our goal for end user's experience

**Through our project, we hope that users will be able to:**

View the social network structure of the characters.
Get the basic statistics of the social network, such as how many characters involved (nodes), how many relationships they have (links) and so on.
Understand the social network node attributes and corresponding social network structures (For instance: Four colleges; Muggle world& Wizarding World, Dark magic& Good Wizard).
Follow the community detection of the social network (four colleges in Hogwarts) and be able to see how the generated community detections differs from the original allocation result.
Know the keywords for each community.
From clustering the sentiment detection/analysis of the characters, understand the general emotional tone of the characters in the book.


**As for the rendering part of the web page (more technical), we hope that we could:**

Implement partitioning of content
Interactive the visualization of social networks(for instance, zoom in and zoom out,etc)
Implement machine learning and data interaction: Predicting which college the character belongs to
Implement data interaction: give a character and the 10 most relevant characters; Or give the soical network associated with the character

<font color=orange>这块是我之前列大纲的时候写的，如果写在这里需要展开+流畅，小点读不太通，没有前因后果</font>


### 2. Basic stats
<!--
* Write about your choices in data cleaning and preprocessing
* Write a short section that discusses the dataset stats (here you can recycle the work you did for Project Assignment A)
-->

<这一块nina整理>

#### 2.1 data cleaning and preprocessing

<font color=orange>这块还是需要注意 **解释**(*每一步: 为什么这么做，具体怎么做的，分析结果*)， 代码保留需要的即可，不要大段打印。</font>

<font color=tomato>(a) **cleaning the node.**  
    the ceritions; (清理原则123，顺便这里可以提一嘴是怎么收集数据的所以需要这样子的清理。)  
    the amount of nodes after cleaning.  
    (b) **cleaning the fandom page**
    掐头去尾+去掉html格式；   
    原因;    
    具体步骤；  
    得到的具体的东西（描述即可不需展示）   
    (c) **书籍的cleaning**
    为什么要这么做；  
    具体步骤；得到的具体的东西（描述即可不需展示）</font>
    
    
    
#### 2.2 basic statiatic of our dataset


<font color=tomato>   
node数量， edge数量，       
    degree分布（注意log-log图+ 分析） ；   
      \gama值；  
    （还有啥能做的不知道了需要再翻翻network science,反正就是能反应网络特性的一些东西）    
   然后记得加分析

</font>


### 3. Tools, theory and analysis. 
<!--
* Talk about how you've worked with text, including regular expressions, unicode, etc.
* Describe which network science tools and data analysis strategies you've used, how those network science measures work, and why the tools you've chosen are right for the problem you're solving.
* How did you use the tools to understand your dataset?
-->

(写得再流畅一点，承上启下那种，逻辑再好点，现在瞎写的)
The questions that we interested about this dataset is：
1. 麻瓜世界与魔法世界的联系。
哈利波特的故事里除了无比强大的巫师们，还有不懂魔法的麻瓜们。对于大多数麻瓜们来说，他们是不知道有魔法世界的存在的，但是麻瓜和魔法世界又有着不可分割的关系。我们想知道：麻瓜世界和魔法世界的是如何连接的？是否有一些关键的结点？两个世界的大小，属性如何？是否有类似于six degree of separation的定律存在于魔法、麻瓜世界 已经两个世界中？ 

2. 四个学院的特征属性。
霍格沃兹有四个学院，在新生入学时，分院帽会将新生分到四个学院中的一个。根据JK罗琳对魔法世界的设定，分院帽会按照学生的特质进行分院——也就是说，每个学院应有自己特有的一些特征。比如，广泛认为，格兰芬多热情勇敢果决。我们想知道通过对fandom页面的文本分析+书的文本分析，来提取每个学院的关键词，得到其属性。

3. 哈利波特世界中人物的情感分析。
作为一个儿童读物，哈利波特中的人物具有鲜明的性格特点和感情倾向，我们很好奇，哪些人物更积极、开心；而哪些人物更加的阴郁、低落？同时我们也相对四个学院的社区进行这样的检测。

4. 利用文本信息的社区检测。
在社区检测方法中，权重值一直被我们所忽略。事实上，在人物A和B，C均认识的情况下，他们之间的关系紧密的强弱并不相同。我们利用文本信息，设计两个节点之间边的权重，再进行社区检测算法。我们将此算法应用在霍格沃兹学院内，讲通过算法产生的分隔与分院帽的结果进行对比，分析之间的不同并探索how smart the sorting hat is.

5. 预测分院结果。
输入一段文字描述，你将会被分到哪一个学院呢？我们希望可以将之前在网络中学习到知识与ML相结合，让哈迷们体验一下自己与哪个学院的特质更相近。

#### 3.1 The connection between magic universe and muggle universe

(解释这个问题，问题的有趣性，我们想要探索的内容blabla)  

(子网络的大小属性)

(子网络的连接关系，有无关键节点，是什么，分析)

(six degree of separation定律的验证和分析)

注意分析解释，想想对面是一个不知道哈利波特的人！

#### 3.2  The most representative words for each house in Hogwarts

问题描述。  

如何获取学院属性。  子网络大小。   

文本分析的具体步骤。

结果分析。与JK罗琳的设定进行对比。


#### 3.3 Sentiment anslysis for characters/houses in Harry Potter network

问题描述。

具体步骤。

结果分析。

对学院的情感分析，分析结果。

#### 3.4 Community detection using text informaion

问题描述。（为什么要这么做要讲清楚）

具体步骤：  
提取文本向量： 具体怎么做的；
相似度：公式；
生成权重；
社区检测（具体算法以及算法如何运行的）

结果分析。与分院帽结果对比。分析分析分析。

#### 3.5 Prediction your house with description test

问题描述。（为什么研究这个 有什么意义）

具体步骤。

结果分析。

### 4. Discussion
<!--
* What went well?,
* What is still missing? What could be improved?, Why?
-->

#### 4.1 Summary on our result

其实我不知道这边还能讲啥。一般discussion可能会说一些延伸的东西。



#### 4.2 Problems and possible improvement

潜在的问题以及可能的解决办法。未来的方向。